In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from azureml.core import Workspace, Dataset

subscription_id = 'b47bd811-d493-4457-b1d3-55f3dd8c85ed'
resource_group = 'DP_100_RG'
workspace_name = 'sentiment-analysis-ws'

workspace = Workspace(subscription_id, resource_group, workspace_name)

train_df = Dataset.get_by_name(workspace, name='train')
train_df = train_df.to_pandas_dataframe()

test_df = Dataset.get_by_name(workspace, name='test')
test_df = test_df.to_pandas_dataframe()

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exce

In [3]:
train_df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [4]:
train_df.iloc[0, 0]

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [5]:
train_df.shape

(40000, 2)

In [6]:
train_df = train_df[:20000]
train_df.label.value_counts()*100/len(train_df)

0    50.12
1    49.88
Name: label, dtype: float64

In [7]:
test_df.shape

(5000, 2)

In [8]:
train_df.label.value_counts()*100/len(train_df)

0    50.12
1    49.88
Name: label, dtype: float64

In [9]:
test_df.label.value_counts()*100/len(train_df)

1    12.525
0    12.475
Name: label, dtype: float64

# Text Preprocessing - Tokenization, Lowering, Stop Word Removal, Removal of non-alphanumeric characters & Lemmatization

In [10]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

def preprocess(line):
    words = nltk.word_tokenize(line) # tokenization
    words = [word.lower() for word in words] # lower casing
    words = [word for word in words if word not in stopwords] # removal of stop words
    words = [word for word in words if word.isalpha()] # removal of non-alphanumeric characters
    words = [lemmatizer.lemmatize(word) for word in words] # lemmatizaion
    return(words)

train_df.text.apply(preprocess)



0        [grew, b, watching, loving, thunderbird, mate,...
1        [put, movie, dvd, player, sat, coke, chip, exp...
2        [people, know, particular, time, past, like, f...
3        [even, though, great, interest, biblical, movi...
4        [im, die, hard, dad, army, fan, nothing, ever,...
                               ...                        
19995    [required, watch, movie, work, pay, contrary, ...
19996    [white, noise, potential, one, talked, movie, ...
19997    [five, deadly, venom, great, action, movie, wr...
19998    [ali, g, indahouse, got, one, funniest, film, ...
19999    [found, six, seven, watching, one, altman, tou...
Name: text, Length: 20000, dtype: object

In [16]:
def pipeline(data):
    X_tfidf = tfidf_vec.transform(data['text'])
    X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vec.get_feature_names_out())
    y_df = data['label']
    return(X_df, y_df)


In [18]:
tfidf_vec = TfidfVectorizer(analyzer=preprocess).fit(train_df['text'])
X_train, y_train = pipeline(train_df)
X_test, y_test = pipeline(test_df)

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [20]:
y_pred = rf.predict(X_test)

In [22]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_train, rf.predict(X_train))
test_acc = accuracy_score(y_test, rf.predict(X_test))

In [23]:
print("Training Accuracy: ", train_acc)
print("Test Accuracy: ", test_acc)

Training Accuracy:  1.0
Test Accuracy:  0.8526


In [24]:
train_df['processed_data'] = train_df.text.apply(preprocess)
test_df['processed_data'] = test_df.text.apply(preprocess)

In [30]:
train_df.to_csv('./processed_train.csv', index=False)

In [31]:
pd.read_csv('processed_train.csv')

,text,label,processed_data
0,I grew up (b. 1965) watching and loving the Th...,0,"['grew', 'b', 'watching', 'loving', 'thunderbi..."
1,"When I put this movie in my DVD player, and sa...",0,"['put', 'movie', 'dvd', 'player', 'sat', 'coke..."
2,Why do people who do not know what a particula...,0,"['people', 'know', 'particular', 'time', 'past..."
3,Even though I have great interest in Biblical ...,0,"['even', 'though', 'great', 'interest', 'bibli..."
4,Im a die hard Dads Army fan and nothing will e...,1,"['im', 'die', 'hard', 'dad', 'army', 'fan', 'n..."
...,...,...,...
19995,"I was required to watch the movie for my work,...",0,"['required', 'watch', 'movie', 'work', 'pay', ..."
19996,"""White Noise"" had potential to be one of the m...",0,"['white', 'noise', 'potential', 'one', 'talked..."
19997,The Five Deadly Venoms is a great kung-fu acti...,1,"['five', 'deadly', 'venom', 'great', 'action',..."
19998,Ali G Indahouse has got to be one of the funni...,1,"['ali', 'g', 'indahouse', 'got', 'one', 'funni..."
